In [1]:
### Load preprocessed data

In [ ]:
#!python ../src/download_ml20.py

In [19]:
import numpy as np
fh = np.load('data/dataset_ml20_wide.npz')
# We have a bunch of feature columns and last column is the y-target
max_seq_len = 768 + 1
train_items = fh['train_items'].astype(np.int64)[:, :max_seq_len]
train_ratng = fh['train_ratng'].astype(np.int64)[:, :max_seq_len]
test_items = fh['test_items'].astype(np.int64)[:, :max_seq_len]
test_ratng = fh['test_ratng'].astype(np.int64)[:, :max_seq_len]

n_user = train_items.shape[0]
n_rank = train_items.shape[1]
n_item = int(train_items.max() + 1)
n_resp = int(train_ratng.max() + 1)

In [40]:
np.unique(train_ratng[train_items > 0])

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [20]:
np.unique(train_items)

array([     0,      2,      3, ..., 131170, 131237, 131262])

In [21]:
!pip install -q reformer_pytorch

In [22]:
import random
import torch
import numpy as np
import pandas as pd
from torch import nn
from torch import from_numpy
import pytorch_lightning as pl
from torch.nn import functional as F
from reformer_pytorch import Reformer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

In [23]:
from abstract_model import AbstractModel

In [41]:
class AR(AbstractModel):
    def __init__(self, n_item, n_dim, n_resp, n_rank, 
                 train_x, train_y, test_x, test_y,
                 heads=2, depth=2, batch_size=32):
        super().__init__()
        self.n_dim = n_dim
        self.n_item = n_item
        self.n_resp = n_resp
        # This means that item=0 will always yield the zero vector
        self.item = nn.Embedding(n_item, n_dim, padding_idx=0)
        self.resp = nn.Embedding(n_resp, n_dim)
        self.reformer = Reformer(dim=n_dim, depth=depth, heads=heads, causal=True, max_seq_len=n_rank)
        self.user_lin = nn.Linear(n_dim, n_dim * n_resp)
        self.item_lin = nn.Linear(n_dim, n_dim * n_resp)
        self.save_data(train_x, train_y, test_x, test_y)
        self.batch_size = batch_size
    
    def forward(self, items, ratng):
        item_vec = self.item(items)
        resp_vec = self.resp(ratng)
        intx_vec = item_vec * resp_vec
        mask = items != -1
        user_vec = self.reformer(intx_vec, input_mask=mask)
        return user_vec
    
    def likelihood(self, user_vec, items, ratg):
        batchsize, window, n_dim = user_vec.shape
        item_vec = self.item(items)
        # Linearly map one user & item vector to one user &  item vector
        # per possible response type
        user_vec_resp = self.user_lin(user_vec).reshape((batchsize, window, n_dim, self.n_resp))
        item_vec_resp = self.item_lin(user_vec).reshape((batchsize, window, n_dim, self.n_resp))
        # Sum interactions  across n_dim back down to  (batchsize, window, n_resp)
        intx = (user_vec_resp * item_vec_resp).sum(dim=2)
        intx_flat = intx.reshape((batchsize * window, self.n_resp))
        ratg_flat = ratg.reshape((batchsize * window))
        loss = F.cross_entropy(intx_flat, ratg_flat, ignore_index=0, reduction='mean')
        return loss
        
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-4, weight_decay=0.01)
    
    def training_step(self, batch, batch_nb):
        (items,), ratng = batch
        # Pass in leading arrays, missing the last element
        # for every user that's to  be predicted
        user_vec = self.forward(items[:,  :-1], ratng[:, :-1])
        # Given previous token, predict the next interaction
        loss = self.likelihood(user_vec, items[:, 1:],  ratng[:, 1:])
        tensorboard_logs = {'train_loss': loss}
        return {'loss': loss, 'log': tensorboard_logs}

    def test_step(self, batch, batch_nb):
        dat = self.training_step(batch, batch_nb)
        dat['test_loss'] = dat.pop('loss')
        return dat

    
n_dim = 16
model = AR(n_item, n_dim, n_resp, n_rank, train_items, train_ratng, test_items, test_ratng)

# add a logger
logger = TensorBoardLogger("tb_logs", name="ar_model")

# We could have turned on multiple GPUs here, for example
# trainer = pl.Trainer(gpus=8, precision=16)    
trainer = pl.Trainer(max_epochs=5,
                     reload_dataloaders_every_epoch=True,
                     logger=logger)    

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


### Train model

In [42]:
trainer.test(model)

KeyboardInterrupt: 

In [38]:
debug

> /opt/conda/lib/python3.6/site-packages/torch/nn/functional.py(1838)nll_loss()
   1836                          .format(input.size(0), target.size(0)))
   1837     if dim == 2:
-> 1838         ret = torch._C._nn.nll_loss(input, target, weight, _Reduction.get_enum(reduction), ignore_index)
   1839     elif dim == 4:
   1840         ret = torch._C._nn.nll_loss2d(input, target, weight, _Reduction.get_enum(reduction), ignore_index)



ipdb>  u


> /opt/conda/lib/python3.6/site-packages/torch/nn/functional.py(2021)cross_entropy()
   2019     if size_average is not None or reduce is not None:
   2020         reduction = _Reduction.legacy_get_string(size_average, reduce)
-> 2021     return nll_loss(log_softmax(input, 1), target, weight, None, ignore_index, None, reduction)
   2022 
   2023 



ipdb>  


> <ipython-input-36-be74371ee154>(39)likelihood()
     37         weig_flat = weig.reshape((batchsize * window))
     38         ratg_flat = ratg.reshape((batchsize * window))
---> 39         loss = F.cross_entropy(intx_flat, ratg_flat, weight=weig_flat, reduction='mean')
     40         return loss
     41 



...

ipdb>  p  ratg_flat


tensor([0, 0, 0,  ..., 0, 0, 0])


ipdb>  p items


tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])


ipdb>  q
